In [ ]:
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle5 as pickle
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
import tensorflow as tf
seed=2022
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/멀캠_2nd_semiproject/데이터/mfcc/Y_data_1516.csv')

In [ ]:
y=df.iloc[:,0]

In [ ]:
with open("/content/drive/MyDrive/멀캠_2nd_semiproject/데이터/mfcc/mfcc_1565_data.pickle","rb") as fr:
    data = pickle.load(fr)

length=[]
for i in range(len(data)):
    length.append(data[i].shape[1])
    
min_len=min(length)

result=[]
for i in range(len(data)):
    result.append(data[i][:,:min_len])
    
result_data=np.array(result)
X=result_data

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=seed
)

In [ ]:
X_train.shape

(1843, 30, 901)

In [28]:
model=Sequential([
       Conv2D(256, kernel_size=(3,3), input_shape=(30, 901, 1), activation='relu'),
       Conv2D(256, (3,3), activation='relu'),
       MaxPooling2D(),    
       Dropout(0.25),
       Conv2D(128, (3,3), activation='relu'),
       Conv2D(128, (3,3), activation='relu'),
       MaxPooling2D(),    
       Flatten(),
       Dense(128, activation='relu'),
       Dropout(0.5),
       Dense(64, activation='relu'),
       Dropout(0.5),
       Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 28, 899, 256)      2560      
                                                                 
 conv2d_19 (Conv2D)          (None, 26, 897, 256)      590080    
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 13, 448, 256)     0         
 g2D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 13, 448, 256)      0         
                                                                 
 conv2d_20 (Conv2D)          (None, 11, 446, 128)      295040    
                                                                 
 conv2d_21 (Conv2D)          (None, 9, 444, 128)       147584    
                                                      

In [29]:
model_path='best-fashion-mnist-cnn-v1.h5'
mc=ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
es=EarlyStopping(patience=10)

In [32]:
model.compile('adam', 'binary_crossentropy', ['accuracy'])

hist=model.fit(
    X_train, y_train, validation_split=0.2, epochs=100, batch_size=50, callbacks=[mc, es]
)

Epoch 1/100
30/30 [==============================] - ETA: 0s - loss: 2.4583 - accuracy: 0.4925
Epoch 1: val_loss improved from inf to 0.69323, saving model to best-fashion-mnist-cnn-v1.h5
30/30 [==============================] - 53s 1s/step - loss: 2.4583 - accuracy: 0.4925 - val_loss: 0.6932 - val_accuracy: 0.4824
Epoch 2/100
30/30 [==============================] - ETA: 0s - loss: 0.6933 - accuracy: 0.4763
Epoch 2: val_loss improved from 0.69323 to 0.69316, saving model to best-fashion-mnist-cnn-v1.h5
30/30 [==============================] - 32s 1s/step - loss: 0.6933 - accuracy: 0.4763 - val_loss: 0.6932 - val_accuracy: 0.4824
Epoch 3/100
30/30 [==============================] - ETA: 0s - loss: 0.6933 - accuracy: 0.4878
Epoch 3: val_loss did not improve from 0.69316
30/30 [==============================] - 32s 1s/step - loss: 0.6933 - accuracy: 0.4878 - val_loss: 0.6932 - val_accuracy: 0.4824
Epoch 4/100
30/30 [==============================] - ETA: 0s - loss: 0.6933 - accuracy: 0.5

In [33]:
best_model=load_model(model_path)
best_model.evaluate(X_test, y_test)

15/15 [==============================] - 6s 263ms/step - loss: 0.6931 - accuracy: 0.4989


[0.6931480765342712, 0.4989154040813446]